## This is an example on how to leverage Picsellia SDK and tensorflow 2 to train a simple Image Classifier 

We suppose that you did not trained a first classifier before and want to start from scratch !

If you want to retrieve a trained model and start from it, please check [this notebook](https://google.com)

In [ ]:
from picsellia import Client 
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import classification_report, confusion_matrix


### Pre-requisite 

🥑 In order to use this notebook, you need to have a **classification dataset in Picsellia** and a **Project** to encapsulate your experiments.

  - You can learn how to create a Dataset [with the Platform](https://documentation.picsellia.com/docs/create-a-dataset) or [with code](https://documentation.picsellia.com/recipes/upload-local-data-to-a-dataset) 

  - Learn how to create a Project and attach dataset [with the Platform](https://documentation.picsellia.com/reference/client#create_project)
  
  - You can find your API Token [here](https://app.picsellia.com/profile#token)


In [ ]:
PICSELLIA_API_TOKEN = "" 
project_name = ""
dataset_name = ""
dataset_version = None
experiment_name = "my-first-classification-experiment"

#### 1 - Let's Initialize Picsellia Client and fetch your project and your dataset

In [ ]:
client = Client(
    api_token=PICSELLIA_API_TOKEN,
)

project = client.get_project(
    name=project_name
)

dataset = client.get_dataset(
    name=dataset_name,
    version=dataset_version
)

#### 2 - Initialize your experiment in order to store and log everything to Picsellia 🥑

In [ ]:
experiment = project.create_experiment(
    name=experiment_name,
    description="First experiment with Image Classification, created by the showcase notebook.",
    dataset=dataset,
)
print(experiment)

#### 3 - Download your Dataset and create the train-test split repartition 📚

`train_test_split`  is recommended to be stored inside Picsellia, this way you will be able to access the validation interface and have full visibility over your training data 👊

If you want to know more about our train_test_split format, here is the [documentation page](https://google.com)


⚠️ You need to download annotations first to download the pictures.

In [ ]:
experiment.dl_annotation()
experiment.dl_picures()
experiment.train_test_split(prop=0.8)

train_split = {
    "x": experiment.categories,
    "y": experiment.train_repartition,
    "image_list": experiment.train_list_id
}
test_split = {
    'x': experiment.categories,
    'y': experiment.test_repartition,
    'image_list': experiment.eval_list_id
}

experiment.log(
    name="train-split",
    data=train_split,
    type="bar",
    replace=True
)

experiment.log(
    name="test-split",
    data=test_split,
    type="bar",
    replace=True
)


#### 4 - Create the Labelmap to and create the right folder tree

`labelmap` is needed for Picsellia to map your verbose labels *(i.e "cat", "dog", "hot-dog")* to your categorical labels *(i.e 1, 2, 3)*.

-> You can find more info about the labelmap format [here](https://google.com)

In [ ]:
labels_index = [e for e in range(1, len(experiment.categories)+1)]

labelmap = dict(zip(labels_index, experiment.categories))

experiment.log(
    name='labelmap', 
    data=labelmap, 
    type='labelmap', 
    replace=True
)

splits = ["train", "validation"]

for split in splits:
    if not split in os.listdir("images"):
        os.mkdir(os.path.join("images", split))
    for category in experiment.categories:
        dirpath = os.path.join("images", split)
        if not os.path.isdir(dirpath):
            os.mkdir(dirpath)

### 5 - Let's move your pictures to the right folders

In [ ]:
train_dir = "images/train"
validation_dir = "images/validation"

for image in experiment.dict_annotations["annotations"]:
    label = image["annotations"][0]["label"]
    image_id = image["internal_picture_id"]
    filename = image["external_picture_url"]
    if image_id in experiment.train_list_id:
        os.rename(os.path.join('images', filename), os.path.join(train_dir, label, filename))
    elif image_id in experiment.eval_list_id:
        os.rename(os.path.join('images', filename), os.path.join(validation_dir, label, filename))

### 6 - Let's create the parameters for your training and send it to Picsellia to retrieve and visualize it on the Platform

🥑 We are also going to create the Dataset from folder thanks to keras method `image_dataset_from_directory()`

In [ ]:
parameters = {
    "batch_size": 4, # You can increase it :D
    "image_size": 64, # We are going to resize the picture 64 x 64
    "learning_rate": 1e5
}

experiment.log(
    name="parameters",
    data=parameters,
    type="table",
    replace=True
)

train_dataset = image_dataset_from_directory(
    directory=train_dir,
    shuffle=True,
    batch_size=parameters["batch_size"],
    image_size=(parameters["image_size"], parameters["image_size"])
)

validation_dataset = image_dataset_from_directory(
    directory=validation_dir,
    shuffle=True,
    batch_size=parameters["batch_size"],
    image_size=(parameters["image_size"], parameters["image_size"])
)

### 7 - Let's visualize some Images and Labels to check everything is okk 😄

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train_dataset.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(train_dataset.class_names[labels[i]])
    plt.axis("off")

#### 8 - Create your Test Dataset and make batches for training 

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

# Let's be really sure that it's alright :D

print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

#### 9 - Configure your Dataset TF config and perform data augmentation 🤔

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.title(train_dataset.class_names[labels[i]])
    plt.axis('off')

#### 10 - Preprocess your Images (rescaling) and define the PicselliaLogger Callback to see your training in real-time 🔬

In [ ]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
IMG_SHAPE = parameters["image_size"] + (3,)


class PicselliaLogger(Callback):

    def on_epoch_end(self, epoch, logs={}):
        for log_name, log_value in logs.items():
            experiment.log(log_name, [float(log_value)], 'line')

callback_list = [PicselliaLogger()]

#### 11 - Define your Classification Model 

**You will create the base model from the MobileNet V2** model developed at Google. 

First, you need to pick which layer of MobileNet V2 you will use for feature extraction. The very last classification layer (on "top", as most diagrams of machine learning models go from bottom to top) is not very useful. 
Instead, you will follow the common practice to depend on the very last layer before the flatten operation. This layer is called the *"bottleneck layer"*. The bottleneck layer features retain more generality as compared to the final/top layer.

First, instantiate a MobileNet V2 model pre-loaded with weights trained on ImageNet. By specifying the include_top=False argument, you load a network that doesn't include the classification layers at the top, which is ideal for feature extraction.

If you want more information about the model used, please check [Keras documentation](https://keras.io/api/applications/mobilenet/#mobilenetv2-function)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=IMG_SHAPE,
    include_top=False,
    weights="imagenet",
)
base_model.trainable = False

# Uncomment if you want a summary of all the layers :D
# base_model.summary() 

image_batch, label_batch = next(iter(train_dataset))
feature_batch = base_model(image_batch)

#### 12 - Add a Classification head

To generate predictions from the block of features, average over the spatial 5x5 spatial locations, using a tf.keras.layers.GlobalAveragePooling2D layer to convert the features to a single 1280-element vector per image.

Then apply a tf.keras.layers.Dense layer to convert these features into a single prediction per image. You don't need an activation function here because this prediction will be treated as a logit, or a raw prediction value. Positive numbers predict class 1, negative numbers predict class 0.

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)


prediction_layer = tf.keras.layers.Dense(1)
prediction_batch = prediction_layer(feature_batch_average)

inputs = tf.keras.Input(shape=(parameters["image_size"], parameters["image_size"], 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

#### 13 - Compile the model and set it to optimize `accuracy` 

In [ ]:
base_learning_rate = parameters["learning_rate"]
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

#### 14 - Launch training of the Dense Layers 🚀 

You need to set the parameters `initial_epochs` to something between 100 and 1000, it's the number of epochs that you will train the Dense Layers alone. 

This parameter is important to store as it indicated how "*long*" you trained your Dense Layers alone before fine-tuning. So let's log it to picsellia :D

In [ ]:
initial_epochs = 100 # Increase it for a real-world use-case
parameters["initial_epochs"] = initial_epochs
experiment.log(
    name="parameters",
    data=parameters,
    type="table",
    replace=False
)

loss0, accuracy0 = model.evaluate(validation_dataset)
print("Initial Loss: {:.2f}".format(loss0))
print("Initial Accuracy: {:.2f}".format(accuracy0))
history = model.fit(
    x=train_dataset,
    epochs=initial_epochs,
    callbacks=callback_list,
    validation_data=validation_dataset
)

#### 15 - Visualize your training / validation accuracy 📈

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

#### 16 - Fine Tune the Top Layers of your models

In the feature extraction experiment, you were only training a few layers on top of an MobileNet V2 base model. The weights of the pre-trained network were not updated during training.

One way to increase performance even further is to train (or "fine-tune") the weights of the top layers of the pre-trained model alongside the training of the classifier you added. The training process will force the weights to be tuned from generic feature maps to features associated specifically with the dataset.

Also, you should try to fine-tune a small number of top layers rather than the whole MobileNet model. In most convolutional networks, the higher up a layer is, the more specialized it is. The first few layers learn very simple and generic features that generalize to almost all types of images. As you go higher up, the features are increasingly more specific to the dataset on which the model was trained. The goal of fine-tuning is to adapt these specialized features to work with the new dataset, rather than overwrite the generic learning.

In [ ]:
base_model.trainable = True
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
    metrics=['accuracy']
)

fine_tune_epochs = 150 # Increase it for a real-world use-case :D
parameters["fine_tune_epochs"] = fine_tune_epochs
experiment.log(
    name="parameters",
    data=parameters,
    type="table",
    replace=False
)

total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(
    x=train_dataset,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1],
    callbacks=callback_list,
    validation_data=validation_dataset
)

#### 17 - Store your trained Model in Picsellia to use it later 🥑

In [ ]:
models_dir = os.path.join(experiment.base_dir, 'models')
tf.saved_model.save(model, os.path.join(models_dir, 'saved_model'))
model.save(os.path.join(models_dir, 'keras_model', 'model.h5'))
experiment.store("keras_model", os.path.join(models_dir, 'keras_model','model.h5'))
experiment.store("model-latest", os.path.join(models_dir, 'saved_model'), zip=True)

#### 18 - Perform Evaluation on your validation dataset and send it to Picsellia to visualize it interactively 🥑

In [ ]:
y_true = []
y_pred = []
for i in range(len(validation_dataset)):
    image_batch, label_batch = validation_dataset.as_numpy_iterator().next()
    predictions = model.predict_on_batch(image_batch).flatten()
    # Apply a sigmoid since our model returns logits
    predictions = tf.nn.sigmoid(predictions)
    predictions = tf.where(predictions < 0.5, 0, 1)
    y_true.extend(label_batch)
    y_pred.extend(predictions.numpy())

confusion_matrix = {
    'categories': experiment.categories,
    'values': confusion_matrix(y_true, y_pred).tolist()
}
experiment.log('confusion-matrix', confusion_matrix, 'heatmap')

# We set a parameters `fine_tune` to True to access it and directly perform fine-tuning without training the first layers next time
parameters["fine_tune"] = True
experiment.log(
    name="parameters",
    data=parameters,
    type="table",
    replace=False
)

experiment.update(status="success")

print(
    """You visualize all your results on Picsellia Reports page :\n
    -> https://app.picsellia.com/{}/experiment/{}/{}/overview
        """.format(client.organization.id, project.id, experiment.id)
)
